In [2]:
import os
from pyspark import SparkConf, SparkContext

from pyspark.sql import SparkSession
import avro.schema
import getopt
import sys
from avro.datafile import DataFileWriter
from avro.io import DatumWriter
from datetime import datetime, date, timedelta
from collections import defaultdict



In [8]:
def getctr(ctr):
    if ctr <= 0.075:
        ctr_level = 0
    elif ctr > 0.075 and ctr <= 0.08730159:
        ctr_level = 1
    elif ctr > 0.08730159 and ctr <= 0.09428571:
        ctr_level = 2
    elif ctr > 0.09428571 and ctr <= 0.09803922:
        ctr_level = 3
    elif ctr > 0.09803922 and ctr <= 0.1015625:
        ctr_level = 4
    elif ctr > 0.1015625 and ctr <=  0.11111111:
        ctr_level = 5
    elif ctr > 0.11111111 and ctr <= 0.11956522:
        ctr_level = 6
    elif ctr > 0.11956522 and ctr <= 0.13461538:
        ctr_level = 7
    elif ctr > 0.13461538 and ctr <= 0.15957447:
        ctr_level = 8
    else:
        ctr_level = 9
    return ctr_level

def getcollect(collect):
    if collect == 0:
        collect_level = 0
    elif collect == 1:
        collect_level = 1
    elif collect > 1 and collect <= 3:
        collect_level = 2
    elif collect > 3 and collect <= 16:
        collect_level = 3
    else:
        collect_level = 4
    return collect_level

def getaddcart(addcart):
    if addcart == 0:
        addcart_level = 0
    elif addcart == 1:
        addcart_level = 1
    elif addcart == 2:
        addcart_level = 2
    elif addcart == 3:
        addcart_level = 3
    else:
        addcart_level = 4
    return addcart_level

def getsale(sale):
    if sale == 0:
        sale_level = 0
    elif sale == 1:
        sale_level = 1
    elif sale == 2:
        sale_level = 2
    elif sale == 2:
        sale_level = 3
    else:
        sale_level = 4
    return sale_level

In [9]:
def main(f, ds):

    spark = SparkSession.builder.appName("lr_sample_train").getOrCreate()
    sql = "select item_id, sale from songwt.item_sale where ds = '%s'" % ds
    print(sql)
    df_sale = spark.sql(sql)
    sale_dict = dict()
    for row in df_sale.collect():
        sale_dict[str(row.item_id)] = row.sale
    #print(sale_dict)
    
    path = "/user/yarn/swt/sample2/%s" % f


    sample = spark.sparkContext.textFile(path)

    lines = sample.collect()
    sellcity = craftsman()
    for line in lines:
        ln = line.replace("(", "")
        ln2 = ln.replace(")", "")
        lnn = ln2.split(",")
        features = dict()
        _l = list()

        if lnn == "null" or lnn == None:
            continue

        for idx, item in enumerate(lnn):
            ft = dict()
            if idx == 0:
                continue
            elif idx == 1:
                ft2 = dict()
                sale = sale_dict.get(item,0)
                if(sale > 0):
                    print(item, sale)
                level = getsale(sale)
                if(level > 0):
                    print(item, level)
                ft2["name"] = "sale"
                ft2["term"] = str(level)
                ft2["value"] = 1.0
                _l.append(ft2)

            elif idx == 2:
                features["label"] = int(item)
            else:
                values = item.split("=")
                name = values[0]
                if name in ["collect2", "add2", "show2", "click2", "city_city", "province"]:
                    continue
                term = values[1].split(":")[0]
                value = values[1].split(":")[1]
                if name == "collect1":
                    new_term = getcollect(float(term))
                elif name == "add1":
                    new_term = getaddcart(float(term))
                elif name.find("city") != -1:
                    city = term.split("_")[1]
                    new_city = sellcity.get(city, "")
                    new_term = term.split("_")[0]+"_"+new_city

                elif name == "show1":
                    exposure = float(term)
                    continue
                elif name == "click1":
                    click = float(term)
                    continue
                else:
                    new_term = term
                ft["name"] = name
                ft["term"] = str(new_term)
                ft["value"] = float(value)
                _l.append(ft)
        ctr = (click+5) / (exposure+50)
        ctr_level = getctr(ctr)
        ctr_dict = dict()
        ctr_dict["name"] = "ctr"
        ctr_dict["value"] = 1.0
        ctr_dict["term"] = str(ctr_level)
        _l.append(ctr_dict)
        ctr_sale_dict = dict()
        ctr_sale_dict["name"] ="ctr_sale"
        ctr_sale_dict["term"] = str(ctr_level)+"_"+str(level)
        ctr_sale_dict["value"] = 1.0
        _l.append(ctr_sale_dict)

        features["features"] = _l
        
        
def craftsman():
    craftsman = dict()
    for ln in open("craftsman.csv", 'rb'):
        ln = ln.strip().decode("utf8")
        lnn = ln.split(",")
        city = lnn[0]
        normalize_city = lnn[3]
        craftsman[city] = normalize_city
    return craftsman
        
        
    
                
                

           
if __name__ == "__main__":
    main("sample_2019-03-20","2019-03-19")
    #test()

select item_id, sale from songwt.item_sale where ds = '2019-03-19'
803383 2
803383 2
1089978 1
1089978 1
791193 1
791193 1
822892 3
822892 4
822892 3
822892 4
756464 5
756464 4
1228843 1
1228843 1
1283979 3
1283979 4
910602 10
910602 4
1149734 2
1149734 2
1054704 68
1054704 4
1235507 1
1235507 1
1054704 68
1054704 4
1109304 12
1109304 4
119336 2
119336 2
843136 1
843136 1
886712 4
886712 4
1218528 6
1218528 4
1279252 1
1279252 1
1336328 1
1336328 1
1311643 10
1311643 4
531262 2
531262 2
1307998 9
1307998 4
1310062 6
1310062 4
143069 10
143069 4
819161 8
819161 4
857957 1
857957 1
1301049 35
1301049 4
1324401 12
1324401 4
1158276 7
1158276 4
803383 2
803383 2
1022498 1
1022498 1
841492 2
841492 2
1099020 21
1099020 4
1125248 7
1125248 4
841103 3
841103 4
856058 15
856058 4
857957 1
857957 1
1327676 1
1327676 1
1099020 21
1099020 4
256387 3
256387 4
865303 6
865303 4
1290254 1
1290254 1
111221 4
111221 4
1265382 2
1265382 2
1278275 15
1278275 4
1201930 4
1201930 4
1330753 18
1330753 4
13

1197298 2
1197298 2
785845 1
785845 1
1299593 2
1299593 2
1301049 35
1301049 4
1158276 7
1158276 4
1248313 3
1248313 4
990628 1
990628 1
1068144 16
1068144 4
1255184 11
1255184 4
1278275 15
1278275 4
796446 2
796446 2
1000238 1
1000238 1
1248313 3
1248313 4
803383 2
803383 2
242817 3
242817 4
242817 3
242817 4
940190 36
940190 4
399814 9
399814 4
691782 8
691782 4
242817 3
242817 4
829617 2
829617 2
1158276 7
1158276 4
242817 3
242817 4
295169 1
295169 1
1307998 9
1307998 4
924360 1
924360 1
1215456 38
1215456 4
1322589 1
1322589 1
1158276 7
1158276 4
803383 2
803383 2
283155 6
283155 4
1290254 1
1290254 1
1036994 1
1036994 1
1325928 16
1325928 4
618176 2
618176 2
664332 5
664332 4
665734 1
665734 1
714926 2
714926 2
1012838 11
1012838 4
242817 3
242817 4
287372 1
287372 1
877094 10
877094 4
822892 3
822892 4
1290559 3
1290559 4
1207593 4
1207593 4
1307998 9
1307998 4
1323045 3
1323045 4
1284249 1
1284249 1
877094 10
877094 4
1109304 12
1109304 4
1294605 8
1294605 4
822892 3
822892 4
1

380614 6
380614 4
458674 14
458674 4
878282 6
878282 4
823988 289
823988 4
827472 108
827472 4
843136 1
843136 1
870116 1
870116 1
920508 1
920508 1
924360 1
924360 1
1015636 3
1015636 4
1118908 1
1118908 1
1284924 10
1284924 4
1312944 22
1312944 4
1253051 3
1253051 4
1259311 10
1259311 4
1158276 7
1158276 4
940144 3
940144 4
1324802 2
1324802 2
435516 2
435516 2
1122792 1
1122792 1
1166732 4
1166732 4
1292620 4
1292620 4
1310576 3
1310576 4
1329419 4
1329419 4
509750 1
509750 1
526750 1
526750 1
1171158 2
1171158 2
1207593 4
1207593 4
877094 10
877094 4
822892 3
822892 4
1225059 1
1225059 1
138686 8
138686 4
1288369 1
1288369 1
138686 8
138686 4
877094 10
877094 4
1158276 7
1158276 4
1192408 5
1192408 4
1310055 7
1310055 4
803383 2
803383 2
785845 1
785845 1
432628 2
432628 2
865275 1
865275 1
48666 1
48666 1
888837 2
888837 2
877094 10
877094 4
242817 3
242817 4
1248313 3
1248313 4
1310062 6
1310062 4
1310055 7
1310055 4
1335812 1
1335812 1
1360024 1
1360024 1
1328471 5
1328471 4
134

1325990 2
1325990 2
664332 5
664332 4
638710 7
638710 4
819082 4
819082 4
1054704 68
1054704 4
1080058 2
1080058 2
1080058 2
1080058 2
867105 18
867105 4
1320801 8
1320801 4
1011036 2
1011036 2
1301596 1
1301596 1
1311035 3
1311035 4
489658 2
489658 2
489658 2
489658 2
1216233 1
1216233 1
1216233 1
1216233 1
1337820 245
1337820 4
1230397 1
1230397 1
1256746 2
1256746 2
957800 1
957800 1
1319472 1
1319472 1
1319472 1
1319472 1
1054704 68
1054704 4
97694 1
97694 1
1337772 1
1337772 1
1335569 4
1335569 4
528348 1
528348 1
1158276 7
1158276 4
1341456 20
1341456 4
681894 14
681894 4
1324401 12
1324401 4
242817 3
242817 4
877094 10
877094 4
1158276 7
1158276 4
840160 4
840160 4
1278275 15
1278275 4
1168742 9
1168742 4
1325445 2
1325445 2
287372 1
287372 1
1277844 1
1277844 1
872437 1
872437 1
872437 1
872437 1
1295649 1
1295649 1
138686 8
138686 4
856058 15
856058 4
1215456 38
1215456 4
1295436 3
1295436 4
1336487 1
1336487 1
618176 2
618176 2
822892 3
822892 4
1125248 7
1125248 4
539720 2
5

143069 4
871159 1
871159 1
1240867 2
1240867 2
1269341 1
1269341 1
1310318 1
1310318 1
310944 2
310944 2
777020 1
777020 1
1029772 2
1029772 2
1161104 6
1161104 4
1255184 11
1255184 4
138686 8
138686 4
622146 5
622146 4
1295649 1
1295649 1
925330 1
925330 1
1347507 1
1347507 1
1320584 1
1320584 1
929474 2
929474 2
1248442 2
1248442 2
877094 10
877094 4
618176 2
618176 2
1054704 68
1054704 4
1109304 12
1109304 4
1158276 7
1158276 4
803383 2
803383 2
1311643 10
1311643 4
877094 10
877094 4
925330 1
925330 1
1307998 9
1307998 4
242817 3
242817 4
111221 4
111221 4
256387 3
256387 4
1358086 1
1358086 1
860696 1
860696 1
792609 2
792609 2
792609 2
792609 2
1207593 4
1207593 4
1330785 2
1330785 2
1100284 3
1100284 4
1330752 1
1330752 1
1273751 3
1273751 4
1307998 9
1307998 4
455188 6
455188 4
1343497 2
1343497 2
1258545 3
1258545 4
1218519 1
1218519 1
1047334 1
1047334 1
1089682 7
1089682 4
1113662 1
1113662 1
430108 1
430108 1
964032 14
964032 4
1248313 3
1248313 4
600666 69
600666 4
714926 

...

793294 2
793294 2
1158276 7
1158276 4
1054704 68
1054704 4
1248442 2
1248442 2
1311643 10
1311643 4
1158276 7
1158276 4
960928 1
960928 1
1054704 68
1054704 4
910470 10
910470 4
1086280 1
1086280 1
1319839 1
1319839 1
161936 8
161936 4
957800 1
957800 1
936334 1
936334 1
1205637 1
1205637 1
1237298 5
1237298 4
1143522 23
1143522 4
803904 2
803904 2
1267135 7
1267135 4
1246539 11
1246539 4
1166878 1
1166878 1
1328692 7
1328692 4
964032 14
964032 4
1266371 20
1266371 4
1171106 2
1171106 2
1319780 1
1319780 1
1332950 9
1332950 4
399606 3
399606 4
1080020 6
1080020 4
1256751 2
1256751 2
32512 1
32512 1
326427 1
326427 1
884536 4
884536 4
1256746 2
1256746 2
1054704 68
1054704 4
1158276 7
1158276 4
1307998 9
1307998 4
1248442 2
1248442 2
1278024 7
1278024 4
960928 1
960928 1
1263269 1
1263269 1
1240277 1
1240277 1
927318 4
927318 4
1277106 2
1277106 2
1158276 7
1158276 4
877094 10
877094 4
1233134 34
1233134 4
760847 18
760847 4
691782 8
691782 4
1321659 1
1321659 1
803383 2
803383 2
966764

618176 2
618176 2
1158276 7
1158276 4
1294605 8
1294605 4
242817 3
242817 4
803383 2
803383 2
184926 1
184926 1
841492 2
841492 2
320786 2
320786 2
1331304 8
1331304 4
1294605 8
1294605 4
920508 1
920508 1
1310707 2
1310707 2
1293231 1
1293231 1
1049694 18
1049694 4
927330 5
927330 4
1215261 6
1215261 4
192086 2
192086 2
664332 5
664332 4
803383 2
803383 2
1307998 9
1307998 4
1307998 9
1307998 4
1158276 7
1158276 4
1258758 92
1258758 4
1313086 1
1313086 1
1286816 26
1286816 4
956726 57
956726 4
242817 3
242817 4
1180252 2
1180252 2
765702 1
765702 1
1342553 1
1342553 1
1109304 12
1109304 4
880107 54
880107 4
877094 10
877094 4
1307998 9
1307998 4
822892 3
822892 4
803383 2
803383 2
1158276 7
1158276 4
857957 1
857957 1
1330950 1
1330950 1
1329195 1
1329195 1
1285362 1
1285362 1
894794 1
894794 1
966690 9
966690 4
911754 1
911754 1
1063610 1
1063610 1
931446 1
931446 1
1271455 2
1271455 2
1295585 2
1295585 2
42774 1
42774 1
1068144 16
1068144 4
1329409 5
1329409 4
403888 2
403888 2
1283

964032 4
1319377 10
1319377 4
1218039 2
1218039 2
792609 2
792609 2
1054704 68
1054704 4
173836 4
173836 4
872181 129
872181 4
1337692 181
1337692 4
1337692 181
1337692 4
1292208 9
1292208 4
1003910 1
1003910 1
489658 2
489658 2
1307998 9
1307998 4
1287128 47
1287128 4
489658 2
489658 2
1214676 136
1214676 4
1225393 1
1225393 1
1197298 2
1197298 2
760321 2
760321 2
348509 4
348509 4
1336328 1
1336328 1
1235507 1
1235507 1
1299433 2
1299433 2
1205637 1
1205637 1
1323550 1
1323550 1
1307445 1
1307445 1
609920 3
609920 4
791193 1
791193 1
1278024 7
1278024 4
1270181 1
1270181 1
1258207 2
1258207 2
1327822 1
1327822 1
910470 10
910470 4
297898 2
297898 2
1235524 1
1235524 1
1326232 1
1326232 1
780757 1
780757 1
1306585 2
1306585 2
516318 2
516318 2
1301049 35
1301049 4
1332137 1
1332137 1
1012236 2
1012236 2
958712 1
958712 1
928830 14
928830 4
1326302 54
1326302 4
1157482 1
1157482 1
198564 1
198564 1
1216206 1
1216206 1
1307998 9
1307998 4
803383 2
803383 2
1310062 6
1310062 4
1341399 5


1341550 1
1128116 1
1128116 1
920508 1
920508 1
1041068 1
1041068 1
910470 10
910470 4
698960 9
698960 4
1253894 1
1253894 1
1131266 4
1131266 4
1228704 3
1228704 4
1233204 2
1233204 2
1323434 1
1323434 1
1306884 1
1306884 1
1323496 1
1323496 1
1256849 2
1256849 2
572892 1
572892 1
841103 3
841103 4
105895 4
105895 4
924360 1
924360 1
1248429 2
1248429 2
458674 14
458674 4
1003182 1
1003182 1
1283740 6
1283740 4
320786 2
320786 2
1011036 2
1011036 2
338048 1
338048 1
1244857 5
1244857 4
652690 1
652690 1
826889 6
826889 4
1273792 5
1273792 4
652892 12
652892 4
960928 1
960928 1
1352887 1
1352887 1
371026 1
371026 1
1287739 2
1287739 2
1214676 136
1214676 4
246564 1
246564 1
1211252 6
1211252 4
1320803 6
1320803 4
405506 4
405506 4
1084128 1
1084128 1
1326302 54
1326302 4
1278275 15
1278275 4
791193 1
791193 1
1233516 6
1233516 4
540942 3
540942 4
888837 2
888837 2
399606 3
399606 4
889651 1
889651 1
1286851 2
1286851 2
878792 2
878792 2
1037864 16
1037864 4
1346121 1
1346121 1
1235524 

1334841 2
1292620 4
1292620 4
1246552 4
1246552 4
1160504 8
1160504 4
100121 1
100121 1
5916 4
5916 4
1221871 4
1221871 4
827472 108
827472 4
785367 2
785367 2
1257729 1
1257729 1
966764 51
966764 4
819161 8
819161 4
871159 1
871159 1
1100284 3
1100284 4
1259311 10
1259311 4
1331942 1
1331942 1
1335569 4
1335569 4
1322538 1
1322538 1
516294 1
516294 1
1237865 1
1237865 1
1322540 2
1322540 2
1143222 1
1143222 1
1215966 1
1215966 1
1211475 1
1211475 1
892179 1
892179 1
718148 1
718148 1
551360 1
551360 1
1230878 27
1230878 4
1233516 6
1233516 4
797991 4
797991 4
1211252 6
1211252 4
111221 4
111221 4
957800 1
957800 1
856058 15
856058 4
1307998 9
1307998 4
803383 2
803383 2
242817 3
242817 4
877094 10
877094 4
1321659 1
1321659 1
326427 1
326427 1
914900 12
914900 4
569914 1
569914 1
591104 1
591104 1
1280384 5
1280384 4
857957 1
857957 1
1221871 4
1221871 4
1335774 4
1335774 4
1164288 1
1164288 1
1226845 3
1226845 4
792734 7
792734 4
951800 19
951800 4
1082174 2
1082174 2
1231210 3
12312